# Analyzing synthetically generated models

In [2]:
import tellurium as te
import teUtils as tu
import os
import csv
from tqdm import tqdm

In [3]:
FOLDER_NAME = '10sp_10n_A/'
DATA_FOLDER = FOLDER_NAME + 'generated_data/'
PERTURBATION_LEVELS = [10, 50] # in percent

## Generating perturbation datasets 

In [4]:
# first create directory to hold the data
os.mkdir('./' + DATA_FOLDER)

In [5]:
with open("passlist.txt") as file:
    passlist = [line.rstrip() for line in file]

Develop perturbation datasets for all models within the FOLDER_NAME.

In [8]:
for pl in PERTURBATION_LEVELS: 
    for i, modelPath in tqdm(enumerate(os.listdir(FOLDER_NAME + 'sbml/'))):
        r = te.loads(FOLDER_NAME + 'sbml/' + modelPath)
        
        exMet = r.getBoundarySpeciesIds()
        inMet = r.getFloatingSpeciesIds()
        fluxnums = range(len(r.getReactionRates()))
        fluxstr = [str(x) for x in fluxnums] 
        fluxIDs = ['flux_' + num for num in fluxstr]
        e_list = [i for i in r.getGlobalParameterIds() if 'E' in i]
        pertLevel = pl/100                

        header = e_list + exMet + inMet + fluxIDs        
        perturbation_level = [1 - pertLevel, 1 + pertLevel]

        with open(DATA_FOLDER+f'{passlist[i]}_pt{pl}.csv', 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            
            try: # base case
                spConc = list(r.simulate(0,1000000)[-1])[1:]
                r.conservedMoietyAnalysis = True
                r.steadyState()
                
                enzymes = [r.getValue(e) for e in e_list]
                exMet_values = [r.getValue(m) for m in exMet]
                fluxes = list(r.getReactionRates())
                
                writer.writerow(enzymes + exMet_values + spConc + fluxes)
            
                # perturbed cases
                for params in e_list:
                    for level in perturbation_level:
                        # print(params, level) ###################
                        r.resetToOrigin()
                        r.setValue(params, level*r.getValue(params))
                        
                        spConc = list(r.simulate(0,1000000)[-1])[1:]
                        r.steadyState()
                        enzymes = [r.getValue(e) for e in e_list]
                        exMet_values = [r.getValue(m) for m in exMet]
                        fluxes = list(r.getReactionRates())
                        
                        writer.writerow(enzymes + exMet_values + spConc + fluxes)
            except:
                pass #print('error')


28it [00:04,  6.68it/s]
28it [00:04,  6.76it/s]


## Plotting analysis of models